<a href="https://colab.research.google.com/github/KutapiAise/COVID-19-Scanner/blob/master/Tensorflow2.0/Tensorflow_2_0_Crash_course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [0]:
data =keras.datasets.imdb

# here the  num_words=10000 picks up the 10000 most frequent words from the imdb dataset, hence shinks thedata
(train_data,train_labels),(test_data,test_labels) = data.load_data(num_words=88000)

In [3]:
print(train_data[7])

[1, 4, 14906, 716, 4, 65, 7, 4, 689, 4367, 6308, 2343, 4804, 28674, 84206, 5270, 32099, 2315, 71688, 12572, 24785, 43394, 4, 10993, 628, 7685, 37, 9, 150, 4, 9820, 4069, 11, 2909, 4, 16287, 847, 313, 6, 176, 63860, 9, 6202, 138, 9, 4434, 19, 4, 96, 183, 26, 4, 192, 15, 27, 5842, 799, 7101, 39455, 588, 84, 11, 4, 3231, 152, 339, 5206, 42, 4869, 30497, 6293, 345, 4804, 37377, 142, 43, 218, 208, 54, 29, 853, 659, 46, 4, 882, 183, 80, 115, 30, 4, 172, 174, 10, 10, 1001, 398, 1001, 1055, 526, 34, 3717, 68395, 5262, 63370, 17, 4, 6706, 1094, 871, 64, 85, 22, 2030, 1109, 38, 230, 9, 4, 4324, 20636, 251, 5056, 1034, 195, 301, 14, 16, 31, 7, 4, 46035, 8, 783, 48545, 33, 4, 2945, 103, 465, 16454, 42, 845, 45, 446, 11, 1895, 19, 184, 76, 32, 4, 5310, 207, 110, 13, 197, 4, 14906, 16, 601, 964, 2152, 595, 13, 258, 4, 1730, 66, 338, 55, 5312, 4, 550, 728, 65, 1196, 8, 1839, 61, 1546, 42, 8361, 61, 602, 120, 45, 7304, 6, 320, 786, 99, 196, 11100, 786, 5936, 4, 225, 4, 373, 1009, 33, 4, 130, 63, 69, 7

In [0]:
word_index=data.get_word_index()

In [0]:
word_index= {k:(v+3) for k,v in word_index.items()}

In [0]:
word_index["<PAD>"] =0
word_index["<START>"] =1
word_index["<UNK>"] =2
word_index["<UNUSED>"] =3

In [0]:
reverse_word_idex=dict([(v,k) for (k,v) in word_index.items()])

In [0]:
def decode_review(text):
  return " ".join([reverse_word_idex.get(i,"?")  for i in text])

In [9]:
print(decode_review(test_data[0]))

<START> please give this one a miss br br kristy swanson and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite lacklustre so all you madison fans give this a miss


In [10]:
# to chek the length of each text data
print(len(train_data[0]), len(train_data[1])) # we see that the input sizes are not definite in all the data texts

218 189


In [0]:
train_data =keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"], padding="post",maxlen=250)
test_data =keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"], padding="post",maxlen=250)

In [12]:
print(len(train_data[0]), len(train_data[1]))

250 250


What is an embedding layer?

conside 2 similar sentences in a context:

0   1   2    3

HAVE A GREAT DAY  

0    1   4   3

HAVE A GOOD DAY

Here "GREAT" and "GOOD" mean the same in a context. However, when these are encoded by th computer, it doesn't know that they mean the same.

what embeddin layer does?
- converts the inputs (10,000  in no) to vectorized form, then group the words that are similar (ie, similar words of the context have small angle b/w them) and ungroup the words that have diferent meaning in the same context.
-the vector created by words have certain dimension (ex 16). So the vectorized  form of the words will have the form of an array with 16 elements where each element denoting magnitude in that particuar unit vector drection.

- vector rep of a word  [0.2 ,0.75,.......,0.8] (16 elements)

- The output of the embedding layer will therefore contain the same number  of values as the input layer with them encoded in the vector format.

What does globalaveragepooling1d do?

- it scals down the input vector in number. for ex. if the number of input is equal to 10,000 then the output of globalavragepooling layer will be lower than that.

What does dense layer do?

it looks for patterens in the word vecors and classify them to pos or neg review




In [13]:
# model down here

model = keras.Sequential()
model.add(keras.layers.Embedding(88000,16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          1408000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,408,289
Trainable params: 1,408,289
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer="adam"  ,loss='binary_crossentropy' , metrics=["accuracy"],)


#validation data

x_val =train_data[:10000]
x_train =train_data[10000:]

y_val =train_labels[:10000]
y_train= train_labels[10000:]

fitModel = model.fit(x_train,y_train, epochs=40, batch_size =512, validation_dat=(x_val,y_val), verbose =1)

results =  model.evaluate(test_data, test_labels)

Epoch 1/40
30/30 [==============================] - 1s 28ms/step - loss: 0.6918 - accuracy: 0.5097
Epoch 2/40
30/30 [==============================] - 1s 27ms/step - loss: 0.6865 - accuracy: 0.6253
Epoch 3/40
30/30 [==============================] - 1s 29ms/step - loss: 0.6738 - accuracy: 0.7665
Epoch 4/40
30/30 [==============================] - 1s 28ms/step - loss: 0.6515 - accuracy: 0.7819
Epoch 5/40
30/30 [==============================] - 1s 28ms/step - loss: 0.6167 - accuracy: 0.8035
Epoch 6/40
30/30 [==============================] - 1s 28ms/step - loss: 0.5711 - accuracy: 0.8291
Epoch 7/40
30/30 [==============================] - 1s 28ms/step - loss: 0.5177 - accuracy: 0.8454
Epoch 8/40
30/30 [==============================] - 1s 28ms/step - loss: 0.4647 - accuracy: 0.8615
Epoch 9/40
30/30 [==============================] - 1s 28ms/step - loss: 0.4162 - accuracy: 0.8784
Epoch 10/40
30/30 [==============================] - 1s 28ms/step - loss: 0.3706 - accuracy: 0.8928
Epoch 11/

In [15]:
print(results) # we get a list of loss, accuracy

[0.3327729105949402, 0.8727999925613403]


In [0]:
model.save("model.h5")

In [0]:
def review_encode(s):
  encoded= [1] # adiing a start tag <START> to be consistant with  the training data

  for word in s:
    if word.lower() in word_index:
      encoded.append(word_index[word.lower()])
    else:
      encoded.append(2)

    return encoded

In [24]:
with open("test.txt", encoding="utf-8") as f:
  for line in f.readlines():
    nline=line.replace(",","").replace(".","").replace("(","").replace(")","").replace(":","").replace("\"","").strip().split()
    encode1 = review_encode(nline)
    encode = keras.preprocessing.sequence.pad_sequences([encode1],value=word_index["<PAD>"], padding="post",maxlen=250)
    predict = model.predict(encode)
    print(nline)
    print(encode1)
    print(encode)
    print(predict[0])

['I', 'liked', 'the', 'first', 'two', 'movies', 'both', 'had', 'a', 'first', 'behind', 'all', 'the', 'action', 'that', 'takes', 'place', 'this', 'movie', 'the', 'plot', 'is', 'weak', 'and', 'the', 'fight', 'scenes', 'way', 'too', 'long', 'for', 'the', 'weak', 'plot', 'Keanu', 'still', 'portrays', 'his', 'character', 'as', 'the', 'Boogeyman', 'really', 'well', 'I', 'enjoyed', 'it', 'but', 'just', 'felt', 'it', 'dragged', 'on', 'a', 'bit', 'much', 'and', 'the', 'fight', 'scene', 'especially', 'with', 'Halle', 'could', 'have', 'been', 'cut', 'down']
[1, 13]
[[ 1 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  

In [18]:
#let us  actualy test some of the predictions made mby the model

test_review = test_data[0]
predict= model.predict([test_review])
print("Review: ")
print(decode_review(test_review))
print("prediction: "+ str(predict[0]))
print("Actual: "+ str(test_labels[0]))

Review: 
<START> please give this one a miss br br <UNK> <UNK> and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite <UNK> so all you madison fans give this a miss <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 

SAVING AND LOADING MODELS


- save the model == done after complete training the model

- checkpoint the mosel == 

 We can also checkpoint the model to save the mdel during its training. This helps the models to resume training at later point of time.
